<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## (Web) Scrape Wikipedia Page on Toronto Neighbourhoods


#### Install Python Packages

In [18]:
#install Beatiful Soup (if not yet available)

!pip install beautifulsoup4

In [19]:
#install xml parser(if not yet available)

!pip install lxml

In [20]:
#install HTML5 library (if not yet available)

!pip install html5lib

#### Import Python Libraries

In [21]:
#import libraries


import pandas as pd
import requests
from bs4 import BeautifulSoup

import html

In [24]:
#get html and 'soupify' table

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')#'html5lib')
#print(soup.prettify)

In [25]:
#access table

table = soup.find('table')
data = pd.read_html(str(table))

In [26]:
#check html to retrieved
data

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns]]

In [27]:
#place table contents into datafrme
df = data[0]

In [28]:
#verify data type

print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [29]:
#display dataframe

df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [30]:
#copy original dataframe in oder to process it
dfp = df

#dfp



#### Build the DataFrame

In [31]:
#set up the dataframe columns for appending

dfp_p = dfp['Postcode']
dfp_b = dfp['Borough']
dfp_n = dfp['Neighbourhood']

In [32]:
#convert series to dataframe

dfp_n = dfp_n.to_frame()
dfp_b = dfp_b.to_frame()
dfp_p = dfp_p.to_frame()

In [33]:
#combine 'slices' into a single dataframe

dfp_bn = dfp_b.merge(dfp_n,left_index=True, right_index=True)
dfp_pbn = dfp_p.merge(dfp_bn,left_index=True, right_index=True)

#### Handle Boroughs that are "Not assigned" 

In [15]:
#remove all Boroughs that are not assigned
dfp_pbn = dfp_pbn[dfp_pbn['Borough'] != "Not assigned"]

print(len(dfp_pbn))

211


#### Handle Neighbourhoods that are "Not assigned" 

In [16]:
#assign Borough value to Neigborhoods that are "Not Assigned"

#initilise variables used in the loop

i=0
nn=0

dfp_pbn_bna = dfp_pbn
rows = len(dfp_pbn_bna)

na = "Not assigned"

#loop through entire dataframe
while i < rows:
    b = dfp_pbn_bna['Borough'].values[i]
    n = dfp_pbn_bna['Neighbourhood'].values[i]
    n.strip()
    b.strip()
    if (n==na):
        nn ++ 1
        print("N: No borough!",i,b,n)
        dfp_pbn_bna['Neighbourhood'].values[i] = b
    i += 1    
        
print(nn,len(dfp_pbn_bna))

N: No borough! 6 Queen's Park Not assigned
0 211


In [17]:
print(dfp_pbn_bna.values[6])

['M7A' "Queen's Park" "Queen's Park"]


#### Handle Neiborhoods that belong to the same Postcode

In [31]:
#join Neighborhoods with the same Postcode

#setup the variables used in the loop
i=0
i2=0
np=0
dfp_pbn_bna_nna = dfp_pbn_bna
rows = len(dfp_pbn_bna_nna)


dfp_pbn_bna_nna.reset_index(drop=True, inplace=True)
#dfp_pbn_bna_nna.head()

dfp_pbn_bna_nna2 = dfp_pbn_bna_nna

#perform a nested loop to concatenate neighborhoods with similar postcodes
while i < rows:
    b = dfp_pbn_bna_nna['Borough'].values[i]
    n = dfp_pbn_bna_nna['Neighbourhood'].values[i]
    p = dfp_pbn_bna_nna['Postcode'].values[i]
    i2 = i + 1
    
    while i2 < rows:
        p2 = dfp_pbn_bna_nna['Postcode'].values[i2]
        if (p == p2): 
            n2 = dfp_pbn_bna_nna['Neighbourhood'].values[i2]
            dfp_pbn_bna_nna['Neighbourhood'].values[i] = n + ", "  +  n2
           
        i2 += 1
    i += 1    
        
print(np,len(dfp_pbn_bna_nna))

0 211


#### Finalise dataframe

In [32]:
dfp_pbn_bna_nna2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [33]:
#remove duplicate rowa
dfp_pbn_bna_nna2.drop_duplicates(subset ="Postcode", keep = 'first', inplace = True) 

In [34]:
print(len(dfp_pbn_bna_nna2))

103


In [35]:
#reset dtaframe to start rows at 0
dfp_pbn_bna_nna2.reset_index(drop=True, inplace=True)

In [36]:
#rename column names
dfp_pbn_bna_nna2.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"},inplace=True)
dfp_pbn_bna_nna2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Final step

In [37]:
#display number of rows and columns
dfp_pbn_bna_nna2.shape

(103, 3)